# Battle of Neighborhoods 
## Getting to know Vancouver, BC Canada

### Installing and importing required libraries

In [1]:
!pip install geocoder
!pip install folium

     |████████████████████████████████| 98 kB 2.0 MB/s 


In [2]:
import pandas as pd
import requests
from bs4 import BeautifulSoup
import geocoder
import folium
from geopy.geocoders import Nominatim 
import numpy as np
from sklearn.cluster import KMeans
import sklearn.cluster.k_means_
import matplotlib.cm as cm
import matplotlib.colors as colors
import matplotlib.pyplot as plt
%matplotlib inline

print('libraries imported successfully')

libraries imported successfully


/usr/local/lib/python3.7/dist-packages/sklearn/utils/deprecation.py:144: FutureWarning: The sklearn.cluster.k_means_ module is  deprecated in version 0.22 and will be removed in version 0.24. The corresponding classes / functions should instead be imported from sklearn.cluster. Anything that cannot be imported from sklearn.cluster is now part of the private API.
  warnings.warn(message, FutureWarning)


#### Web Scraping and Data Cleaning

In [3]:
url='https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_V'
data = requests.get(url).text

In [4]:
soup = BeautifulSoup(data, 'html5lib')

In [5]:
table_contents=[]
table=soup.find('table')
for row in table.findAll('td'):
    #print(row.span.text)
    cell = {}
    if row.span.text=='Not assigned':
        pass
    else:
        cell['PostalCode'] = row.b.text[:3]
        #print(cell['PostalCode'])
        cityNeighbor = (row.span.text).split('(')
        cell['City'] = cityNeighbor[0]
        #print(cell['City'])
        if len(cityNeighbor) > 1:
          cell['Neighborhood'] = ((((cityNeighbor[1]).strip(')')).replace(' /',',')).replace(')',' ')).strip(' ')
        else:
          cell['Neighborhood'] = cell['City']
        #print(cell['Neighborhood'])
        table_contents.append(cell)

df=pd.DataFrame(table_contents)

In [6]:
df

,PostalCode,City,Neighborhood
0,V1A,Kimberley,Kimberley
1,V2A,Penticton,Penticton
2,V3A,Langley Township,Langley City
3,V4A,SurreySouthwest,SurreySouthwest
4,V5A,Burnaby,"Government Road, Lake City, SFU, Burnaby Mountain"
...,...,...,...
170,V5Z,Vancouver,"East Fairview, South Cambie"
171,V6Z,Vancouver,SW Downtown
172,V7Z,Sechelt,Halfmoon Bay
173,V8Z,Victoria,Middle Saanich


In [7]:
print(len(df.City.unique()))
df.City.unique()

115


array(['Kimberley', 'Penticton', 'Langley Township', 'SurreySouthwest',
       'Burnaby', 'Vancouver', 'RichmondSouth', 'Powell River',
       'Victoria', 'VernonEast', 'KamloopsNorthwest',
       'Port CoquitlamCentral', 'White Rock', 'Richmond', 'Squamish',
       'Cranbrook', 'KamloopsCentral and Southeast',
       'Port CoquitlamSouth', 'DeltaNortheast', 'RichmondNorthwest',
       'Kitimat', 'Salmon Arm', 'KamloopsSouth and West',
       'CoquitlamNorth', 'DeltaEast', 'RichmondSouthwest', 'Whistler',
       'Dawson Creek', 'Williams Lake', 'AbbotsfordEast',
       'DeltaEast Central', 'North Vancouver ', 'Terrace', 'Ladysmith',
       'VernonWest', 'KamloopsNorth', 'Port Moody',
       'Campbell RiverOutskirts', 'Fort St. John', 'Quesnel',
       'Prince Rupert', 'CourtenayNorthern Outskirts', 'Merritt',
       'Prince GeorgeNorth', 'CoquitlamSouth', 'DeltaNorthwest',
       'Salt Spring Island', 'Qualicum Beach', 'Nelson',
       'Prince GeorgeEast Central', 'New WestminsterNorth

In [8]:
df.describe()

,PostalCode,City,Neighborhood
count,175,175,175
unique,175,115,174
top,V5W,Vancouver,CoquitlamNorth
freq,1,31,2


In [9]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 175 entries, 0 to 174
Data columns (total 3 columns):
 #   Column        Non-Null Count  Dtype 
---  ------        --------------  ----- 
 0   PostalCode    175 non-null    object
 1   City          175 non-null    object
 2   Neighborhood  175 non-null    object
dtypes: object(3)
memory usage: 4.2+ KB


#### Getting Langituteds and Longitudes for each Postal Area

In [10]:
df["Latitude"] = ""
df["Longitude"] = ""
df.shape

(175, 5)

In [11]:
#Function to get latitude and longitude using geocoder
def get_latilong(postal_code):
    lati_long_coords = None
    while(lati_long_coords is None):
        g = geocoder.arcgis('{}, Vancouver, BC'.format(postal_code))
        lati_long_coords = g.latlng
    return lati_long_coords
    
get_latilong('V5A')

[49.260380000000055, -123.11335999999994]

In [12]:
#Adding Lat Long for each Postal Code to the dataframes
for i, row in df.iterrows():
    lat_lng_coords = get_latilong(df.iloc[i,0])
    df.at[i,'Latitude'] = lat_lng_coords[0]
    df.at[i,'Longitude'] = lat_lng_coords[1]
    print(str(i)+'/'+str(df.shape[0]),end='\r')

In [13]:
df.head()

,PostalCode,City,Neighborhood,Latitude,Longitude
0,V1A,Kimberley,Kimberley,49.2604,-123.113
1,V2A,Penticton,Penticton,49.2604,-123.113
2,V3A,Langley Township,Langley City,49.2604,-123.113
3,V4A,SurreySouthwest,SurreySouthwest,49.2604,-123.113
4,V5A,Burnaby,"Government Road, Lake City, SFU, Burnaby Mountain",49.2604,-123.113


#### Mapping Vancouver

In [14]:
address = 'Vancouver, British Columbia Canada'
geolocator = Nominatim(user_agent="ny_explorer")
location = geolocator.geocode(address)
latitude = location.latitude
longitude = location.longitude
print('The geograpical coordinate of Vancouver Canada are {}, {}.'.format(latitude, longitude))

The geograpical coordinate of Vancouver Canada are 49.2608724, -123.1139529.


In [15]:
map_Van = folium.Map(location=[latitude, longitude], zoom_start=12.5)
for lat, lng, city, neighborhood in zip(df['Latitude'], df['Longitude'], df['City'], df['Neighborhood']):
    label = '{}, {}'.format(neighborhood, city)
    label = folium.Popup(label, parse_html=True)
    folium.CircleMarker(
        [lat, lng],
        radius=4,
        popup=label,
        color='blue',
        fill=True,
        fill_color='#87cefa',
        fill_opacity=0.5,
        parse_html=False).add_to(map_Van)
map_Van

In [16]:
CLIENT_ID = 'TUVZ3WINAY2HYXHVANOTWL2XYLZULVCWXTMQXPG4GKQYIGJX' # your Foursquare ID
CLIENT_SECRET = 'HAOZ03RX0E3T0VZOIGX52R5EDOUZU5ZS13XPH0BVFI3PU3UA' # your Foursquare Secret
VERSION = '20210822' # Foursquare API version
LIMIT = 50 # A default Foursquare API limit value

print('Your credentails:')
print('CLIENT_ID: ' + CLIENT_ID)
print('CLIENT_SECRET:' + CLIENT_SECRET)

Your credentails:
CLIENT_ID: TUVZ3WINAY2HYXHVANOTWL2XYLZULVCWXTMQXPG4GKQYIGJX
CLIENT_SECRET:HAOZ03RX0E3T0VZOIGX52R5EDOUZU5ZS13XPH0BVFI3PU3UA


In [17]:
#Function to get nearby shops in a Postal Area within a radius of 15km
def getNearbyShops(cities, neighbors, latitudes, longitudes, query='shopping'):
    
    shops_list=[]
    for city, neighbor, lat, lng in zip(cities, neighbors, latitudes, longitudes):
        
        url = 'https://api.foursquare.com/v2/venues/search?&client_id={}&client_secret={}&v={}&ll={},{}&query={}&radius={}&limit={}'.format(
            CLIENT_ID, CLIENT_SECRET, VERSION, lat, lng, query, 10000, LIMIT)
        
        results = requests.get(url).json()["response"]['venues']

        if city==neighbor:
          name = city
        else:
          name = neighbor+', '+city
        
        shops_list.append([(name, s['name'], s['location']['lat'], s['location']['lng'], s['location']['distance'], ' '.join(map(str,s['location']['formattedAddress'])), s['categories'][0]['name']) for s in results])

    nearby_shops = pd.DataFrame([item for shop_list in shops_list for item in shop_list])
    nearby_shops.columns = ['Neighborhood', 'ShopName', 'Latitude', 'Longitude', 'Distance (in km)', 'Address','Category']
    
    return(nearby_shops)

In [18]:
url = 'https://api.foursquare.com/v2/venues/search?&client_id={}&client_secret={}&v={}&ll={},{}&query={}&limit={}'.format(
            CLIENT_ID, CLIENT_SECRET, VERSION, 49.2604,-123.113, 'shopping', LIMIT)
        
results = requests.get(url).json()["response"]
results

{'venues': [{'categories': [{'icon': {'prefix': 'https://ss3.4sqi.net/img/categories_v2/shops/mall_',
      'suffix': '.png'},
     'id': '4bf58dd8d48988d1fd941735',
     'name': 'Shopping Mall',
     'pluralName': 'Shopping Malls',
     'primary': True,
     'shortName': 'Mall'}],
   'hasPerk': False,
   'id': '4aa7fd69f964a5208e4e20e3',
   'location': {'address': '555 W 12th Ave',
    'cc': 'CA',
    'city': 'Vancouver',
    'country': 'Canada',
    'crossStreet': 'btwn Cambie & Ash',
    'distance': 263,
    'formattedAddress': ['555 W 12th Ave (btwn Cambie & Ash)',
     'Vancouver BC V5Z 3X7',
     'Canada'],
    'labeledLatLngs': [{'label': 'display',
      'lat': 49.26094664620503,
      'lng': -123.11653525528322}],
    'lat': 49.26094664620503,
    'lng': -123.11653525528322,
    'postalCode': 'V5Z 3X7',
    'state': 'BC'},
   'name': 'City Square Shopping Centre',
   'referralId': 'v-1629863022'},
  {'categories': [{'icon': {'prefix': 'https://ss3.4sqi.net/img/categories_v2/sh

In [19]:
VanShops = getNearbyShops(cities=df['City'],neighbors=df['Neighborhood'],latitudes=df['Latitude'],longitudes=df['Longitude'])

#### Exploring Shops in Vancouver

In [20]:
VanShops['Distance (in km)'] = VanShops['Distance (in km)']/1000
print(VanShops.shape)
VanShops.head()

(3618, 7)


,Neighborhood,ShopName,Latitude,Longitude,Distance (in km),Address,Category
0,Kimberley,City Square Shopping Centre,49.260947,-123.116535,0.239,555 W 12th Ave (btwn Cambie & Ash) Vancouver B...,Shopping Mall
1,Kimberley,Kensington Square Shopping Centre,49.279939,-122.968162,10.768,6500 E Hastings St. (at Kensington Ave.) Burna...,Shopping Mall
2,Kimberley,Centrepoint Shopping Mall,49.259030,-123.101381,0.883,2949 Main St (at E 13th Ave) Vancouver BC V5T ...,Shopping Mall
3,Kimberley,MaisonBirks - Park Royal Shopping Centre,49.325820,-123.138045,7.501,1015 Park Royal Street West Vancouver BC V7T 1...,Jewelry Store
4,Kimberley,New York Fries - Park Royal Shopping Centre,49.326261,-123.138991,7.566,2002 Park Royal S (Marine Drive) West Vancouve...,Restaurant


In [21]:
VanShops.groupby('Neighborhood').count()

,ShopName,Latitude,Longitude,Distance (in km),Address,Category
Neighborhood,,,,,,
AbbotsfordEast,21,21,21,21,21,21
AbbotsfordSoutheast,21,21,21,21,21,21
AbbotsfordSouthwest,21,21,21,21,21,21
AbbotsfordWest,21,21,21,21,21,21
"Bentall Centre, Vancouver",18,18,18,18,18,18
...,...,...,...,...,...,...
"district municipality Inner East, North Vancouver",21,21,21,21,21,21
"district municipality North Central, North Vancouver",21,21,21,21,21,21
"district municipality Northwest Central, North Vancouver",21,21,21,21,21,21


In [22]:
print('There are {} unique Shopping categories.'.format(len(VanShops['Category'].unique())))

There are 11 unique Shopping categories.


In [23]:
VanShops.describe()

,Latitude,Longitude,Distance (in km)
count,3618.000000,3618.000000,3618.000000
mean,49.268225,-123.076045,6.877082
std,0.048118,0.067698,3.604866
min,49.141929,-123.254195,0.238000
25%,49.232138,-123.118965,3.157000
50%,49.279939,-123.116535,7.529000
75%,49.316769,-123.005344,9.023000
max,49.350731,-122.852727,14.803000


In [24]:
VanShops.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 3618 entries, 0 to 3617
Data columns (total 7 columns):
 #   Column            Non-Null Count  Dtype  
---  ------            --------------  -----  
 0   Neighborhood      3618 non-null   object 
 1   ShopName          3618 non-null   object 
 2   Latitude          3618 non-null   float64
 3   Longitude         3618 non-null   float64
 4   Distance (in km)  3618 non-null   float64
 5   Address           3618 non-null   object 
 6   Category          3618 non-null   object 
dtypes: float64(3), object(4)
memory usage: 198.0+ KB


In [25]:
VanShops['ShopName'].groupby(pd.cut(VanShops['Distance (in km)'],np.arange(0,16,1))).count()

Distance (in km)
(0, 1]      305
(1, 2]       16
(2, 3]      466
(3, 4]      333
(4, 5]       44
(5, 6]       66
(6, 7]       61
(7, 8]      784
(8, 9]      628
(9, 10]     202
(10, 11]    214
(11, 12]    175
(12, 13]    163
(13, 14]    152
(14, 15]      9
Name: ShopName, dtype: int64

In [26]:
VanShops['Distance Range'] = pd.cut(VanShops['Distance (in km)'],np.arange(0,16,1))
VanShops.head()

,Neighborhood,ShopName,Latitude,Longitude,Distance (in km),Address,Category,Distance Range
0,Kimberley,City Square Shopping Centre,49.260947,-123.116535,0.239,555 W 12th Ave (btwn Cambie & Ash) Vancouver B...,Shopping Mall,"(0, 1]"
1,Kimberley,Kensington Square Shopping Centre,49.279939,-122.968162,10.768,6500 E Hastings St. (at Kensington Ave.) Burna...,Shopping Mall,"(10, 11]"
2,Kimberley,Centrepoint Shopping Mall,49.259030,-123.101381,0.883,2949 Main St (at E 13th Ave) Vancouver BC V5T ...,Shopping Mall,"(0, 1]"
3,Kimberley,MaisonBirks - Park Royal Shopping Centre,49.325820,-123.138045,7.501,1015 Park Royal Street West Vancouver BC V7T 1...,Jewelry Store,"(7, 8]"
4,Kimberley,New York Fries - Park Royal Shopping Centre,49.326261,-123.138991,7.566,2002 Park Royal S (Marine Drive) West Vancouve...,Restaurant,"(7, 8]"


In [27]:
VanShopsHot = pd.get_dummies(VanShops[['Distance Range']], prefix="Distance_km", prefix_sep="")
VanShopsHot.insert(0,'Neighborhood',VanShops['Neighborhood'])
print(VanShopsHot.shape)
VanShopsHot.head()

(3618, 16)


,Neighborhood,"Distance_km(0, 1]","Distance_km(1, 2]","Distance_km(2, 3]","Distance_km(3, 4]","Distance_km(4, 5]","Distance_km(5, 6]","Distance_km(6, 7]","Distance_km(7, 8]","Distance_km(8, 9]","Distance_km(9, 10]","Distance_km(10, 11]","Distance_km(11, 12]","Distance_km(12, 13]","Distance_km(13, 14]","Distance_km(14, 15]"
0,Kimberley,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0
1,Kimberley,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0
2,Kimberley,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0
3,Kimberley,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0
4,Kimberley,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0


In [28]:
VanShopsHotGrouped = VanShopsHot.groupby('Neighborhood').mean().reset_index()
print(VanShopsHotGrouped.shape)
VanShopsHotGrouped.head()

(174, 16)


,Neighborhood,"Distance_km(0, 1]","Distance_km(1, 2]","Distance_km(2, 3]","Distance_km(3, 4]","Distance_km(4, 5]","Distance_km(5, 6]","Distance_km(6, 7]","Distance_km(7, 8]","Distance_km(8, 9]","Distance_km(9, 10]","Distance_km(10, 11]","Distance_km(11, 12]","Distance_km(12, 13]","Distance_km(13, 14]","Distance_km(14, 15]"
0,AbbotsfordEast,0.095238,0.0,0.142857,0.095238,0.000000,0.000000,0.000000,0.238095,0.190476,0.047619,0.047619,0.047619,0.047619,0.047619,0.000000
1,AbbotsfordSoutheast,0.095238,0.0,0.142857,0.095238,0.000000,0.000000,0.000000,0.238095,0.190476,0.047619,0.047619,0.047619,0.047619,0.047619,0.000000
2,AbbotsfordSouthwest,0.095238,0.0,0.142857,0.095238,0.000000,0.000000,0.000000,0.238095,0.190476,0.047619,0.047619,0.047619,0.047619,0.047619,0.000000
3,AbbotsfordWest,0.095238,0.0,0.142857,0.095238,0.000000,0.000000,0.000000,0.238095,0.190476,0.047619,0.047619,0.047619,0.047619,0.047619,0.000000
4,"Bentall Centre, Vancouver",0.166667,0.0,0.055556,0.055556,0.222222,0.055556,0.166667,0.000000,0.000000,0.000000,0.055556,0.111111,0.055556,0.000000,0.055556


In [29]:
NumMostShops = 5
for neigh in VanShopsHotGrouped['Neighborhood']:
    print(neigh+':')
    temp = VanShopsHotGrouped[VanShopsHotGrouped['Neighborhood'] == neigh].T.reset_index()
    temp.columns = ['Distance Range','freq']
    temp = temp.iloc[1:]
    print(temp.sort_values('freq', ascending=False).reset_index(drop=True).head(NumMostShops))
    print('\n')

AbbotsfordEast:
      Distance Range       freq
0  Distance_km(7, 8]   0.238095
1  Distance_km(8, 9]   0.190476
2  Distance_km(2, 3]   0.142857
3  Distance_km(0, 1]  0.0952381
4  Distance_km(3, 4]  0.0952381


AbbotsfordSoutheast:
      Distance Range       freq
0  Distance_km(7, 8]   0.238095
1  Distance_km(8, 9]   0.190476
2  Distance_km(2, 3]   0.142857
3  Distance_km(0, 1]  0.0952381
4  Distance_km(3, 4]  0.0952381


AbbotsfordSouthwest:
      Distance Range       freq
0  Distance_km(7, 8]   0.238095
1  Distance_km(8, 9]   0.190476
2  Distance_km(2, 3]   0.142857
3  Distance_km(0, 1]  0.0952381
4  Distance_km(3, 4]  0.0952381


AbbotsfordWest:
      Distance Range       freq
0  Distance_km(7, 8]   0.238095
1  Distance_km(8, 9]   0.190476
2  Distance_km(2, 3]   0.142857
3  Distance_km(0, 1]  0.0952381
4  Distance_km(3, 4]  0.0952381


Bentall Centre, Vancouver:
        Distance Range       freq
0    Distance_km(4, 5]   0.222222
1    Distance_km(0, 1]   0.166667
2    Distance_km(6, 7

In [30]:
def DistanceShops(row, maxshops):
    row_distances = row.iloc[1:]
    row_distances_sorted = row_distances.sort_values(ascending=False)
    return row_distances_sorted.index.values[0:maxshops]

In [31]:
ShopsSorted = pd.DataFrame(columns=['Neighborhood','MostNumShops','2ndMostNumShops','3rdMostNumShops','4thMostNumShops','5thMostNumShops'])
ShopsSorted['Neighborhood'] = VanShopsHotGrouped['Neighborhood']
for ind in np.arange(VanShopsHotGrouped.shape[0]):
    ShopsSorted.iloc[ind, 1:] = DistanceShops(VanShopsHotGrouped.iloc[ind, :], maxshops=5)
print(ShopsSorted.shape)
ShopsSorted.head()

(174, 6)


,Neighborhood,MostNumShops,2ndMostNumShops,3rdMostNumShops,4thMostNumShops,5thMostNumShops
0,AbbotsfordEast,"Distance_km(7, 8]","Distance_km(8, 9]","Distance_km(2, 3]","Distance_km(3, 4]","Distance_km(0, 1]"
1,AbbotsfordSoutheast,"Distance_km(7, 8]","Distance_km(8, 9]","Distance_km(2, 3]","Distance_km(3, 4]","Distance_km(0, 1]"
2,AbbotsfordSouthwest,"Distance_km(7, 8]","Distance_km(8, 9]","Distance_km(2, 3]","Distance_km(3, 4]","Distance_km(0, 1]"
3,AbbotsfordWest,"Distance_km(7, 8]","Distance_km(8, 9]","Distance_km(2, 3]","Distance_km(3, 4]","Distance_km(0, 1]"
4,"Bentall Centre, Vancouver","Distance_km(4, 5]","Distance_km(6, 7]","Distance_km(0, 1]","Distance_km(11, 12]","Distance_km(14, 15]"


In [32]:
kclusters = 5
VanShopsHotGroupedCluster = VanShopsHotGrouped.drop('Neighborhood', 1)
km = KMeans(n_clusters=kclusters, init='k-means++', max_iter=100, n_init=1,verbose=True,random_state=1)
kmeans = km.fit(VanShopsHotGroupedCluster)
print(kmeans.labels_[0:10])
print(len(kmeans.labels_))

Initialization complete
start iteration
done sorting
end inner loop
Iteration 0, inertia 1.8865663709893115
start iteration
done sorting
end inner loop
Iteration 1, inertia 1.7978573367474397
start iteration
done sorting
end inner loop
Iteration 2, inertia 1.7978573367474397
center shift 0.000000e+00 within tolerance 1.802393e-07
[0 0 0 0 4 0 0 0 0 0]
174


In [33]:
ShopsSorted.insert(0, 'Cluster Labels', kmeans.labels_)
VanMerged = df
VanMerged = VanMerged.join(ShopsSorted.set_index('Neighborhood'), on='Neighborhood')

In [34]:
print(VanMerged.shape)
VanMerged.head()

(175, 11)


,PostalCode,City,Neighborhood,Latitude,Longitude,Cluster Labels,MostNumShops,2ndMostNumShops,3rdMostNumShops,4thMostNumShops,5thMostNumShops
0,V1A,Kimberley,Kimberley,49.2604,-123.113,0.0,"Distance_km(7, 8]","Distance_km(8, 9]","Distance_km(2, 3]","Distance_km(3, 4]","Distance_km(0, 1]"
1,V2A,Penticton,Penticton,49.2604,-123.113,0.0,"Distance_km(7, 8]","Distance_km(8, 9]","Distance_km(2, 3]","Distance_km(3, 4]","Distance_km(0, 1]"
2,V3A,Langley Township,Langley City,49.2604,-123.113,NaN,NaN,NaN,NaN,NaN,NaN
3,V4A,SurreySouthwest,SurreySouthwest,49.2604,-123.113,0.0,"Distance_km(7, 8]","Distance_km(8, 9]","Distance_km(2, 3]","Distance_km(3, 4]","Distance_km(0, 1]"
4,V5A,Burnaby,"Government Road, Lake City, SFU, Burnaby Mountain",49.2604,-123.113,NaN,NaN,NaN,NaN,NaN,NaN


In [35]:
map_clusters = folium.Map(location=[latitude, longitude], zoom_start=11)

x = np.arange(kclusters)
ys = [i+x+(i*x)**2 for i in range(kclusters)]
colors_array = cm.rainbow(np.linspace(0, 1, len(ys)))
rainbow = [colors.rgb2hex(i) for i in colors_array]

markers_colors = []
for lat, lon, poi, cluster in zip(VanMerged['Latitude'], VanMerged['Longitude'], VanMerged['Neighborhood'],kmeans.labels_):
    label = folium.Popup(str(poi) + ' Cluster ' + str(cluster), parse_html=True)
    folium.CircleMarker([lat, lon], radius=5, popup=label, color=rainbow[cluster-1], fill=True, fill_color=rainbow[cluster-1], fill_opacity=0.7).add_to(map_clusters)
map_clusters